<a href="https://colab.research.google.com/github/akbln/en_fr_translation_test_deployment/blob/main/Ahmad_Kabalan_Gradio_EN_FR_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the model and tokenizers
import gdown

file_ids = {
    'model.keras': '1LVr5ranuhkWijWdatpqmPkIT8-hAg2yq',
    'english_tokenizer.json': '1BmezVYQjo7GyWhoTr7p2mOfK3_PenpIt',
    'french_tokenizer.json': '1-w5CloLXg0PnMn14-Pgoxwe1xlLWs-sG'
}

# Download and save each file in the Colab environment
for filename, file_id in file_ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, filename, quiet=True)
    print(f"Downloaded {filename} successfully!")


Downloaded model.keras successfully!
Downloaded english_tokenizer.json successfully!
Downloaded french_tokenizer.json successfully!


In [2]:
# Define the model as a class
import numpy as np
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.saving import load_model

class TranslationModel:
    def __init__(self, model_path, eng_tokenizer_path, fr_tokenizer_path, max_eng_length, max_fr_length):
        self.model_path = model_path
        self.eng_tokenizer_path = eng_tokenizer_path
        self.fr_tokenizer_path = fr_tokenizer_path
        self.max_eng_length = max_eng_length
        self.max_fr_length = max_fr_length
        self.model = None
        self.eng_tokenizer = None
        self.fr_tokenizer = None

        # Load the model and tokenizers
        self.load_model()
        self.load_tokenizers()

    def load_model(self):
        self.model = load_model(self.model_path)


    def load_tokenizers(self):

        with open(self.eng_tokenizer_path, "r") as file:
            self.eng_tokenizer = tokenizer_from_json(json.load(file))
        with open(self.fr_tokenizer_path, "r") as file:
            self.fr_tokenizer = tokenizer_from_json(json.load(file))
        print("Tokenizers loaded successfully.")


    def translate_sentence(self, eng_sentence):
        if not self.model or not self.eng_tokenizer or not self.fr_tokenizer:
            return "Model or tokenizers not loaded."

        eng_sequence = self.eng_tokenizer.texts_to_sequences([eng_sentence])
        eng_sequence = pad_sequences(eng_sequence, maxlen=self.max_eng_length, padding='post')

        # Generate predictions
        predictions = self.model.predict(eng_sequence)

        # Convert predictions to French words
        fr_sentence = []
        for time_step in predictions[0]:
            word_index = np.argmax(time_step)
            if word_index == 0:  # Stop at the padding or end token
                break
            word = self.fr_tokenizer.index_word.get(word_index, '')
            if word:
                fr_sentence.append(word)

        return ' '.join(fr_sentence)


In [ ]:
!pip install gradio

In [4]:
import gradio as gr

model = TranslationModel(model_path="./model.keras",
                         eng_tokenizer_path="./english_tokenizer.json",
                         fr_tokenizer_path="./french_tokenizer.json",
                         max_fr_length=21,
                         max_eng_length=15
                         )
def greet(english_sentence):
    return model.translate_sentence(english_sentence)

interface = gr.Interface(
    fn=greet,
    inputs = gr.Textbox(label="English sentence: "),
    outputs = gr.Textbox(label="French Translation:"),
    allow_flagging="never",
    submit_btn = gr.Button(value="Translate")
)

interface.launch()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 17 variables whereas the saved optimizer has 32 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Tokenizers loaded successfully.


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e5b2f133d1d238d8d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
